In [1]:
import h5py
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

In [2]:
def load_dataset():
    train_dataset = h5py.File('./data/train_catvnoncat.h5', 'r')
    train_images = np.array(train_dataset["train_set_x"][:]) # your train set features
    train_labels = np.array(train_dataset["train_set_y"][:]) # your train set labels

    test_dataset = h5py.File('./data/test_catvnoncat.h5', 'r')
    test_images = np.array(test_dataset["test_set_x"][:]) # your test set features
    test_labels = np.array(test_dataset["test_set_y"][:]) # your test set labels

    classes = np.array(test_dataset["list_classes"][:]) # the list of classes
    
    train_labels = train_labels.reshape((1, train_labels.shape[0]))
    test_labels = test_labels.reshape((1, test_labels.shape[0]))
    
    return train_images, train_labels, test_images, test_labels, classes

In [3]:
# Load training and test data
train_images, train_labels, test_images, test_labels, classes = load_dataset()

# Normalising images and reshape labels
train_images = train_images / 255.0
test_images = test_images / 255.0

train_labels = train_labels.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)

# Converting data to PyTorch tensors
train_images_tensor = torch.tensor(train_images, dtype=torch.float32).view(train_images.shape[0], -1)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_images_tensor = torch.tensor(test_images, dtype=torch.float32).view(test_images.shape[0], -1)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)

# Creating data loaders
batch_size = 64
train_data = TensorDataset(train_images_tensor, train_labels_tensor)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)

test_data = TensorDataset(test_images_tensor, test_labels_tensor)
test_loader = DataLoader(test_data, batch_size=batch_size)

In [9]:
import torch.nn as nn

class MultiLayerNN(nn.Module):
    def __init__(self, input_size, sizes):
        super(MultiLayerNN, self).__init__()

        # Define the layers based on the sizes list fc1 = fully connected layer 1
        self.fc1 = nn.Linear(input_size, sizes[0])
        self.fc2 = nn.Linear(sizes[0], sizes[1])
        self.fc3 = nn.Linear(sizes[1], sizes[2])
        self.fc4 = nn.Linear(sizes[2], sizes[3])
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return self.sigmoid(x)

input_size = train_images.shape[1] * train_images.shape[2] * train_images.shape[3]
sizes = [64, 32, 64, 1]
model = MultiLayerNN(input_size, sizes)


In [10]:
loss_fn = nn.BCELoss()  # binary cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.0015)


In [11]:
num_epochs = 25
for epoch in range(num_epochs):
    for batch_idx, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = loss_fn(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [1/25], Loss: 0.6225
Epoch [2/25], Loss: 0.6559
Epoch [3/25], Loss: 0.5675
Epoch [4/25], Loss: 0.8466
Epoch [5/25], Loss: 0.6117
Epoch [6/25], Loss: 0.5461
Epoch [7/25], Loss: 0.6338
Epoch [8/25], Loss: 0.4889
Epoch [9/25], Loss: 0.5383
Epoch [10/25], Loss: 0.5303
Epoch [11/25], Loss: 0.5590
Epoch [12/25], Loss: 0.6505
Epoch [13/25], Loss: 0.5271
Epoch [14/25], Loss: 0.4695
Epoch [15/25], Loss: 0.4289
Epoch [16/25], Loss: 0.4504
Epoch [17/25], Loss: 0.4748
Epoch [18/25], Loss: 0.5404
Epoch [19/25], Loss: 0.4098
Epoch [20/25], Loss: 0.3628
Epoch [21/25], Loss: 0.3450
Epoch [22/25], Loss: 0.3855
Epoch [23/25], Loss: 0.5454
Epoch [24/25], Loss: 0.2792
Epoch [25/25], Loss: 0.4200
